## *** Fun picture ***

# Tanzanian Water Well Analysis

**Authors:** Juan Acosta, Drew Holcombe, and Raul Torres
***

# Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

# Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

# Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix, recall_score,\
    accuracy_score, precision_score, f1_score
from sklearn.tree import DecisionTreeClassifier, plot_tree

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

In [2]:
! ls ".\data"

training_set_labels.csv
training_set_values.csv


Our data is split between two files - labels and values. The source also included a "test set"; however, this set did not contain the wells' functionality, our target variable. As such, we'll treat the "training data" as our full dataset.

In [3]:
train_values = pd.read_csv("./data/training_set_values.csv")
train_labels = pd.read_csv("./data/training_set_labels.csv")

In [4]:
train_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [5]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


The data will be easiest to work with if we merge the two.

In [6]:
df = train_values.merge(train_labels)

The columns are described [here](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/25/), but the information provided is far from exhaustive, and many terms are not defined. We'll have to explore what we can work with, and what we're unable to understand and, by extension, present.

# Data Preparation

Nulls are found throughout the dataset, which will have to be handled on a case-by-case basis. We'll also have to deal with variables that are not defined in a way that we can understand.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [8]:
# Date recorded is formatted as an object - we'll convert it into a
# timestamp so we can work with it more easily.
df["date_recorded"] = pd.to_datetime(df["date_recorded"])

In [9]:
df["status_group"].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

## Columns Lacking Usable/Useful Information

In [10]:
drop_columns = []

In [11]:
df["id"].value_counts()

2047     1
72310    1
49805    1
51852    1
62091    1
        ..
46396    1
36155    1
34106    1
38200    1
0        1
Name: id, Length: 59400, dtype: int64

These unique IDs are not helpful, as we already have the index of the df anyway.

The date_recorded and recorded_by also wouldn't be contributing factors; we'll drop this as well.

In [12]:
drop_columns.extend(["date_recorded", "recorded_by"])

In [13]:
df["public_meeting"].value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

This column is described only as "true/false." As we cannot report on it, this column will need to be dropped from our analysis.

In [14]:
drop_columns.append("public_meeting")

In [15]:
df['num_private'].value_counts()

0      58643
6         81
1         73
5         46
8         46
       ...  
180        1
213        1
23         1
55         1
94         1
Name: num_private, Length: 65, dtype: int64

This column also has no description from the source, so we'll drop it as well.

In [16]:
drop_columns.append("num_private")

In [17]:
df["wpt_name"].value_counts()

none                  3563
Shuleni               1748
Zahanati               830
Msikitini              535
Kanisani               323
                      ... 
Area Two Namba 18        1
Boma Saving              1
Mch. Julias              1
Fanuel Akyoo             1
Kanisani La Mitume       1
Name: wpt_name, Length: 37400, dtype: int64

This column indicates specific waterpoints, but with so many unique waterpoints represented in the set (waterpoints with only one well), it won't give our model useful information.

In [18]:
drop_columns.append("wpt_name")

In [19]:
df["installer"].isna().sum()

3655

In [20]:
list(df["installer"].unique())

['Roman',
 'GRUMETI',
 'World vision',
 'UNICEF',
 'Artisan',
 'DWE',
 'DWSP',
 'Water Aid',
 'Private',
 'DANIDA',
 'Lawatefuka water sup',
 'WEDECO',
 'Danid',
 'TWE',
 'ISF',
 'Kilolo Star',
 'District council',
 'Water',
 'WU',
 nan,
 'Not known',
 'Central government',
 'CEFA',
 'Commu',
 'Accra',
 'World Vision',
 'LGA',
 'MUWSA',
 'KKKT _ Konde and DWE',
 'Government',
 'Olgilai village community',
 'KKKT',
 'RWE',
 'Adra /Community',
 'SEMA',
 'SHIPO',
 'HESAWA',
 'ACRA',
 'Community',
 'IFAD',
 'Sengerema Water Department',
 'HE',
 'ISF and TACARE',
 'Kokeni',
 'DA',
 'Adra',
 'ALLYS',
 'AICT',
 'KIUMA',
 'CES',
 'District Counci',
 'Ruthe',
 'Adra/Community',
 'Tulawaka Gold Mine',
 'KKT C',
 'Hesawa',
 'Water board',
 'LOCAL CONTRACT',
 'WFP',
 'LIPS',
 'TASAF',
 'World',
 '0',
 'SW',
 'Shipo',
 'Fini water',
 'Kanisa',
 'OXFARM',
 'VILLAGE COUNCIL Orpha',
 'Villagers',
 'Idara ya maji',
 'FPCT',
 'WVT',
 'Ir',
 'DANID',
 'Angli',
 'secondary school',
 'Amref',
 'JBG',
 'DAD

Unfortunately, the installer column is not well-formatted. We see a number of versions of the same concept (particularly versions of "unknown," "government," and various school groups). As such, while this could be useful information, we do not have time to clean this data and will have to drop this column. With additional time and resources, this could be worth revisiting. 

In [21]:
drop_columns.append("installer")

In [22]:
df["funder"].isna().sum()

3635

In [23]:
df["funder"].value_counts()

Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
                          ... 
Kaaya                        1
Hez                          1
Mikumi G                     1
Hery                         1
Ded_rwsp                     1
Name: funder, Length: 1897, dtype: int64

In [24]:
list(df["funder"].unique())

['Roman',
 'Grumeti',
 'Lottery Club',
 'Unicef',
 'Action In A',
 'Mkinga Distric Coun',
 'Dwsp',
 'Rwssp',
 'Wateraid',
 'Isingiro Ho',
 'Private',
 'Danida',
 'World Vision',
 'Lawatefuka Water Supply',
 'Biore',
 'Rudep',
 'Hesawa',
 'Twe',
 'Isf',
 'African Development Bank',
 'Government Of Tanzania',
 'Sobodo',
 'Water',
 'Private Individual',
 'Undp',
 nan,
 'Not Known',
 'Kirde',
 'Cefa',
 'Ces(gmbh)',
 'European Union',
 'Lga',
 'District Council',
 'Muwsa',
 'Dwe/norad',
 'Kkkt_makwale',
 'Sawaka',
 'Ces (gmbh)',
 'Olgilai Village Community',
 'Kkkt',
 'Roman Catholic',
 'Norad',
 'Adra',
 'Sema',
 'Piusi',
 'Dwe',
 'Rc Church',
 'Swisland/ Mount Meru Flowers',
 'Ifad',
 'Swedish',
 'Idc',
 'He',
 'Isf/tacare',
 'Jica',
 'Mzee Sh',
 'Aict',
 'Tcrs',
 'Kiuma',
 'Germany Republi',
 'Netherlands',
 'Ruthe',
 'Tulawaka Gold Mine',
 'Nethalan',
 'Tasaf',
 'Concern World Wide',
 'Wfp',
 'Lips',
 'Sida',
 'World Bank',
 'Tanza',
 '0',
 'Sw',
 'Shipo',
 'Fini Water',
 'Kanisa',
 'Ox

As with the "installer" column, we do not have the time or resources to parse through the messy data in this column. As such, we'll drop this one as well.

In [25]:
drop_columns.append("funder")

In [26]:
print(df["management"].value_counts())

print('-----------------------------')

print(df['management_group'].value_counts())

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64
-----------------------------
user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64


These columns, which explore the same concept with differing levels of specificity, are not well-defined in our source material. Barring understanding of these terms and acronyms, we'll have to drop them from our model - we will not be able to describe them intelligently without additonal information. As such, these will also be dropped from our model.

In [27]:
drop_columns.extend(["management", "management_group"])

In [28]:
df["permit"].isna().sum()

3056

In [29]:
df["permit"].value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [30]:
df.loc[df['permit'] == True]["status_group"].value_counts(normalize=True)

functional                 0.554437
non functional             0.376145
functional needs repair    0.069417
Name: status_group, dtype: float64

In [31]:
df.loc[df['permit'] == False]["status_group"].value_counts(normalize=True)

functional                 0.517094
non functional             0.407443
functional needs repair    0.075463
Name: status_group, dtype: float64

In [32]:
df.loc[(df['permit'] != True)&(df['permit'] != False)]["status_group"].value_counts(normalize=True)

functional                 0.547448
non functional             0.354385
functional needs repair    0.098168
Name: status_group, dtype: float64

In [33]:
df.loc[(df['permit'] != True)&(df['permit'] != False), "permit"] = "Unknown"

In [34]:
df["permit"].value_counts()

True       38852
False      17492
Unknown     3056
Name: permit, dtype: int64

The wells with missing data in this column are unlike both the permitted and unpermitted wells: they are more likely to be in need of repair than either other set, while they have the fewest non-functional wells. As such, we'll use this as a distinct third group.

## Columns With Similar Descriptions

We have a large number of columns indicating the location of the wells. Let's look at them all:

In [35]:
print(df["region_code"].value_counts())

print('-----------------------------')

print(df['district_code'].value_counts())

11    5300
17    5011
12    4639
3     4379
5     4040
18    3324
19    3047
2     3024
16    2816
10    2640
4     2513
1     2201
13    2093
14    1979
20    1969
15    1808
6     1609
21    1583
80    1238
60    1025
90     917
7      805
99     423
9      390
24     326
8      300
40       1
Name: region_code, dtype: int64
-----------------------------
1     12203
2     11173
3      9998
4      8999
5      4356
6      4074
7      3343
8      1043
30      995
33      874
53      745
43      505
13      391
23      293
63      195
62      109
60       63
0        23
80       12
67        6
Name: district_code, dtype: int64


With no documentation of what these numbers correspond to, we'll need to drop these columns.

In [36]:
drop_columns.extend(["region_code", "district_code"])

 Let's explore the other location descriptions.

In [37]:
print(df['basin'].value_counts())

print('-----------------------------')

print(df['subvillage'].value_counts())

print('-----------------------------')

print(df['region'].value_counts())

print('-----------------------------')

print(df['lga'].value_counts())

print('-----------------------------')

print(df['ward'].value_counts())

Lake Victoria              10248
Pangani                     8940
Rufiji                      7976
Internal                    7785
Lake Tanganyika             6432
Wami / Ruvu                 5987
Lake Nyasa                  5085
Ruvuma / Southern Coast     4493
Lake Rukwa                  2454
Name: basin, dtype: int64
-----------------------------
Madukani    508
Shuleni     506
Majengo     502
Kati        373
Mtakuja     262
           ... 
Namilop       1
Wansambo      1
Ndwita        1
Namiyodi      1
Katende       1
Name: subvillage, Length: 19287, dtype: int64
-----------------------------
Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730

Given these are describing the same thing (the location) with varrying levels of specificity, we'll need to narrow these down to a single column.

Based on background research, while the subvillages are the most specific of the location data, they are rather unweildy; many cannot be found by a google search. As such, we'll use the slightly less-specific, but more wieldy "ward" column.

Coordinates for individual wells are also given; as we do not have tools to work with these meaningfully, we'll drop them from the set.

In [38]:
drop_columns.extend(["lga", "latitude", "longitude", "subvillage", "region", "basin"])

There are a number of pairs or small groups of columns with identical descriptions, similar to the location. Let's look through them; we'll be keeping the most specific columns.

In [39]:
print(df['water_quality'].value_counts())

print('-----------------------------')

print(df['quality_group'].value_counts())

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64
-----------------------------
good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64


The two have almost identical data; the main difference is the division of the "salty abandoned" and "flouride abandoned" columns in the water_quality column. We'll drop this column in favor of the water_quality column.

In [40]:
drop_columns.append("quality_group")

Similarly, quantity and quantity_group also have identical descriptions.

In [41]:
print(df['quantity'].value_counts())

print('-----------------------------')

print(df['quantity_group'].value_counts())

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64
-----------------------------
enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64


These are identical, so we'll arbitrarily drop quantity_group.

In [42]:
drop_columns.append("quantity_group")

In [43]:
print(df['source'].value_counts())

print('-----------------------------')

print(df['source_type'].value_counts())

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64
-----------------------------
spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64


We'll keep source since it distinguishes a handful of values that are grouped together in from source_type:
   1. "other" is split into "other" and "unknown" 
   2. "river/lake" is split into "river" and "lake"
   3. "borehole" is split into "hand dtw" and "machine dbh"

In [44]:
drop_columns.append("source_type")

In [45]:
print(df['waterpoint_type'].value_counts())

print('-----------------------------')

print(df['waterpoint_type_group'].value_counts())

communal standpipe             28522
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64
-----------------------------
communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64


We'll keep the more specific classifications of waterpoint_type. 

In [46]:
drop_columns.append("waterpoint_type_group")

In [47]:
print(df['extraction_type'].value_counts())

print('-----------------------------')

print(df['extraction_type_group'].value_counts())

print('-----------------------------')

print(df['extraction_type_class'].value_counts())

gravity                      26780
nira/tanira                   8154
other                         6430
submersible                   4764
swn 80                        3670
mono                          2865
india mark ii                 2400
afridev                       1770
ksb                           1415
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  98
cemo                            90
other - play pump               85
walimi                          48
climax                          32
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64
-----------------------------
gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
indi

The most specific column is extraction_type; we'll drop the other two.

In [48]:
drop_columns.extend(["extraction_type_group", "extraction_type_class"])

In [49]:
print(df['payment'].value_counts())

print('-----------------------------')

print(df['payment_type'].value_counts())

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64
-----------------------------
never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64


These two are identical except for having slightly different naming conventions, so we'll arbitrarily drop payment_type.

In [50]:
drop_columns.append("payment_type")

## Columns with Missing Values

In [51]:
df['construction_year'].value_counts().head()

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
Name: construction_year, dtype: int64

This column is missing nearly half its values, so we'll drop it.

In [52]:
drop_columns.append("construction_year")

In [53]:
df['population'].value_counts().head()

0      21381
1       7025
200     1940
150     1892
250     1681
Name: population, dtype: int64

This is also a large number of missing values; additionally, there are a large number of populations of "1" which are highly implausible. Futher, the area the population number refers to is unclear. We'll drop this column as well.

In [54]:
drop_columns.append("population")

In [55]:
df["amount_tsh"].value_counts()

0.0         41639
500.0        3102
50.0         2472
1000.0       1488
20.0         1463
            ...  
8500.0          1
6300.0          1
220.0           1
138000.0        1
12.0            1
Name: amount_tsh, Length: 98, dtype: int64

In [56]:
df.loc[df["amount_tsh"] == 0]["status_group"].value_counts()

functional                 19706
non functional             18885
functional needs repair     3048
Name: status_group, dtype: int64

Interestingly, we have a large number of wells that supposedly have no available water, yet over half of them are listed as being functional. Given the rest of the dataset often uses 0 in place of unknown values, this column seems unreliable. As such, we'll have to drop this column.

In [57]:
drop_columns.append("amount_tsh")

In [58]:
df["gps_height"].value_counts()

 0       20438
-15         60
-16         55
-13         55
-20         52
         ...  
 2285        1
 2424        1
 2552        1
 2413        1
 2385        1
Name: gps_height, Length: 2428, dtype: int64

Similarly to amount_tsh, we don't have a good way to work around which values of 0 are authentic and which are actually nulls. As such, while this feature could be interesting, we'll have to drop it.

In [59]:
drop_columns.append("gps_height")

In [60]:
print(df["scheme_management"].isna().sum())

print('-----------------------------')

print(df['scheme_name'].isna().sum())

3877
-----------------------------
28166


Due to having so many nulls in a categorical variable, we'll have to delete these columns.

In [61]:
drop_columns.extend(["scheme_management", "scheme_name"])

In [63]:
drop_columns

['date_recorded',
 'recorded_by',
 'public_meeting',
 'num_private',
 'wpt_name',
 'installer',
 'funder',
 'management',
 'management_group',
 'region_code',
 'district_code',
 'lga',
 'latitude',
 'longitude',
 'subvillage',
 'region',
 'basin',
 'quality_group',
 'quantity_group',
 'source_type',
 'waterpoint_type_group',
 'extraction_type_group',
 'extraction_type_class',
 'payment_type',
 'construction_year',
 'population',
 'amount_tsh',
 'gps_height',
 'scheme_management',
 'scheme_name']

# Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [62]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***